# CUDA

Throughout the book, we've mostly been using PyTorch or tools built on top of it, such as fastai and Hugging Face transformers. When we first introduced it in this book, we pitched PyTorch as a low-level framework, where you build architectures and write training loops "from scratch" using your knowledge of linear algebra.

But PyTorch may not be the lowest level of abstraction you deal with in machine learning.

PyTorch itself is written in C++, to which the CUDA language is an extension. CUDA is self-described as a "programming model" that allows you to write code for Nvidia GPUs. When writing your C++ code, you include certain functions called "CUDA Kernels" that perform a portion of the work on the GPU.

> Who's That Pokemon? CUDA Kernels

> A kernel is a function that is compiled for and designed to run on special accelerator hardware like GPUs (graphic processing units), FPGAs (field-programmable gate arrays), and ASICs (application-specific integated circuits). They are generally written by engineers who are very familiar with the hardware architecture, and are extensively tuned to perform a single task very well, such as matrix multiplication or convolution. CUDA kernels are kernels run on devices that use CUDA-Nvidia's GPUs and accelerators.

PyTorch and many other deep learning frameworks use a handful of CUDA kernels to implement their backend, and then build a higher-level interface to a language like Python. This allows you to run super-fast, hand-tuned code on specialized hardware that experts have spent years optimizing without having to think about memory, pointers, threads, etc.

There are many ohter similiar platforms, like AMS's ROCm, SYCL (an open source alternative from the Khronos Group), and, with AI hardware startups showing up in every nook and corner, many more.

But CUDA is, by far, the most mature and well-developed GPU programming interface available today. In fact, it's mostly the reason that we're all forced to use Nvidia's GPUs-its software stack is just so much better than everyone else's, which makes it easier to develop libraries like PyTorch on top of it.

Unless you have the bandwidth, it's not always a great idea to look for kerne-level improvements. This is probably very low on the list of things you should do if youre focus is on deploying an NLP application using existing tools and technology.

But...it is useful to understand how such a critical component of the infrastructure that powers deep learning today works, and it's certainly interesting and fun. An understanding of some of the idears in CUDA may also help you debug obscure errors in you deep learning framework, and can help you make more informed purchasing decisions for hardware.

## Threads and Thread Blocks

## Writing CUDA Kernels

## CUDA in Practice